<p align="center">
    <img src="https://github.com/GeostatsGuy/GeostatsPy/blob/master/TCG_color_logo.png?raw=true" width="220" height="240" />

</p>

## Interactive Gibbs Sampler 

### Michael J. Pyrcz, Professor, The University of Texas at Austin 

*Novel Data Analytics, Geostatistics and Machine Learning Subsurface Solutions*

#### Gibbs Sampler

I teach the Gibbs Sampler as part of my lecture on Markov chain Monte Carlo (McMC) methods. This is critical to understand solution methods Bayesian machine learning methods. See my lectures:

* [Bayesian linear regression lecture](https://youtu.be/LzZ5b3wdZQk?si=3Uu2pvCjsl1fH5qU)
* [Markov chain Monte Carlo](https://youtu.be/7QX-yVboLhk?si=o7CSimpgFhjT1Vxo)
* [Bayesian Linear Regression Example](https://youtu.be/JG69fxKzwt8?si=ywn9xC_Pe8YQwR2f)

Gibbs sampler is one of the most intuitive methods for McMC.

* as usual we don't have access to the joint distribution, but we have access to the conditional distributions. 
* instead of sampleing directly from the joint distribution (not available), we sequentially sample from the conditional distribution! 

For a bivariate example, features $X_1$ and $X_2$, we proceed as follows:

1. Assign random values for $𝑋_1^{\ell=0}$, $X_2^{\ell=0}$
<p></p>
2. Sample from $𝑓(𝑋_1|X_2^{\ell=0})$ to get $𝑋_1^{\ell=1}$ 
<p></p>
3. Sample from $𝑓(𝑋_2|X_1^{\ell=1})$ to get $𝑋_2^{\ell=1}$ 
<p></p>
4. Repeat for the next steps / samples, $\ell = 1,\ldots,𝐿$

Although we only applied the conditional distribution, the resulting samples will have the correct joint distribution.

\begin{equation}
f(X_1,X_2)
\end{equation}

We never needed to use the joint distribution, we only needed the conditionals!

* Bayesian Linear Regression - we apply Gibbs sampler to sample the posterior distributions of the model parameters given the data.

#### Gibbs Sampler for Bivariate Gaussian Distribution

Below I build out an interactive Gibbs sampler to sample the bivariate joint Gaussian distribution from only the conditional distributions!

#### Load and Configure the Required Libraries

The following code loads the required libraries and sets a plotting default.

In [1]:
%matplotlib inline
supress_warnings = False
import os                                               # to set current working directory 
import sys                                              # supress output to screen for interactive variogram modeling
import numpy as np                                      # arrays and matrix math
import pandas as pd                                     # DataFrames
from scipy.stats import norm                            # Gaussian PDF
import matplotlib.pyplot as plt                         # plotting
import seaborn as sns                                   # plot PDF
from sklearn.model_selection import train_test_split    # train and test split
from sklearn import tree                                # tree program from scikit learn (package for machine learning)
from sklearn import metrics                             # measures to check our models
import scipy.spatial as spatial                         #search for neighbours
from matplotlib.patches import Rectangle                # build a custom legend
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator) # control of axes ticks
import math                                             # sqrt operator
from ipywidgets import interactive                      # widgets and interactivity
from ipywidgets import widgets                            
from ipywidgets import Layout
from ipywidgets import Label
from ipywidgets import VBox, HBox
cmap = plt.cm.inferno                                   # default color bar, no bias and friendly for color vision defeciency
plt.rc('axes', axisbelow=True)                          # grid behind plotting elements
if supress_warnings == True:
    import warnings                                     # supress any warnings for this demonstration
    warnings.filterwarnings('ignore')                  

#### Declare Functions

The following functions for clean code. 

* Just a improved grid for the plot.

In [2]:
def add_grid():
    plt.gca().grid(True, which='major',linewidth = 1.0); plt.gca().grid(True, which='minor',linewidth = 0.2) # add y grids
    plt.gca().tick_params(which='major',length=7); plt.gca().tick_params(which='minor', length=4)
    plt.gca().xaxis.set_minor_locator(AutoMinorLocator()); plt.gca().yaxis.set_minor_locator(AutoMinorLocator()) # turn on minor ticks

#### Interactive Gibbs Sampler to Sample the Bivariate Gausian Distribution Dashboard

Here's a dashboard with a cool visualization for my interactive Gibbs sampler.

In [3]:
l = widgets.Text(value='                                  Interactive Gibbs Sampler Demo, Prof. Michael Pyrcz, The University of Texas at Austin',
                 layout=Layout(width='750px', height='30px'))

nsample = widgets.IntSlider(min=1, max = 101, value=10, step = 1, description = '$n_{sample}$',orientation='horizontal', 
           style = {'description_width': 'initial'},layout=Layout(width='370px', height='30px'),continuous_update=False)
rho = widgets.FloatSlider(min=-1.0, max = 1.0, value=0.7, step = 0.1, description = r'$\rho_{X_1,X_2}$',orientation='horizontal',
           style = {'description_width': 'initial'},layout=Layout(width='370px', height='30px'),continuous_update=False)

ui = widgets.HBox([nsample,rho],)
ui2 = widgets.VBox([l,ui],)

def run_plot(nsample,rho):
    mu1 = 0.0; sig1 = 1.0; mu2 = 0.0; sig2 = 1.0; seed = 73073; nc = 200
    
    L = nsample
    np.random.seed(seed=seed)
    x1 = np.zeros(L); x2 = np.zeros(L); x = np.linspace(-3,3,nc)
    
    x1[0] = np.random.rand(1) * 6.0 - 3.0; x2[0] = np.random.rand(1) * 6.0 - 3.0; 
    
    plt.subplot(111)
    plt.scatter(x1[0],x2[0],color='grey',edgecolor='black',s=15,zorder=4)
    
    case = 0
    
    for l in range(1,L):
        if case == 0: # update x2
            x1[l] = x1[l-1]
            lmu = mu2 + rho * (sig2/sig1) * (x1[l] - mu1); lstd = 1 - rho**2
            x2[l] = np.random.normal(loc = lmu,scale = lstd,size = 1)
            case = 1
            plt.scatter(x1[l],x2[l],color='blue',edgecolor='black',s=15,alpha=1.0,zorder=100)
            plt.plot([x1[l-1],x1[l]],[x2[l-1],x2[l]],color='black',lw=1,alpha = max((l-(L-20))/20,0),zorder=4)
            plt.plot([x1[l-1],x1[l]],[x2[l-1],x2[l]],color='white',lw=3,alpha = max((l-(L-20))/20,0),zorder=3)
            if l == L-1:
                #plt.plot([x1[l],x1[l]],[-3,3],color='blue',alpha=0.7,zorder=10)
                pdf = norm.pdf(x, loc=lmu, scale=lstd)*0.5
                mask = pdf > np.percentile(pdf,q=40)
                plt.fill_betweenx(x[mask],x1[l]+pdf[mask],np.full(len(x[mask]),x1[l]),color='blue',alpha=0.2,zorder=2)
                plt.plot(x1[l]+pdf[mask],x[mask],color='blue',alpha=0.7,zorder=1)
                plt.arrow(x1[l-1],x2[l-1],0,x2[l]-x2[l-1],color='black',lw=0.5,head_width=0.05,length_includes_head=True,zorder=100)
                plt.scatter(x1[l],x2[l],color='white',edgecolor='blue',s=30,linewidth=1,alpha=1.0,zorder=100)
                plt.annotate(r'$f_{X_2|X_1}$ = ' + str(np.round(x1[l],2)),xy=[x1[l]+0.02,max(x[mask])-0.2],color='blue',rotation=-90)
        elif case == 1: # update x1
            x2[l] = x2[l-1]
            lmu = mu1 + rho * (sig1/sig2) * (x2[l] - mu2); lstd = 1 - rho**2
            x1[l] = np.random.normal(loc = lmu,scale = lstd,size = 1)
            case = 0
            plt.scatter(x1[l],x2[l],color='red',edgecolor='black',s=15,alpha=1.0,zorder=100)
            plt.plot([x1[l-1],x1[l]],[x2[l-1],x2[l]],color='black',lw=1,alpha = max((l-(L-20))/20,0),zorder=4)
            plt.plot([x1[l-1],x1[l]],[x2[l-1],x2[l]],color='white',lw=3,alpha = max((l-(L-20))/20,0),zorder=3)
            if l == L-1:
                #plt.plot([-3,3],[x2[l],x2[l]],color='red',alpha=0.7,zorder=10)
                pdf = norm.pdf(x, loc=lmu, scale=lstd)*0.5
                mask = pdf > np.percentile(pdf,q=40)
                plt.fill_between(x[mask],x2[l]+pdf[mask],np.full(len(x[mask]),x2[l]),color='red',alpha=0.2,zorder=2)
                plt.plot(x[mask],x2[l]+pdf[mask],color='red',alpha=0.7,zorder=1)
                plt.arrow(x1[l-1],x2[l-1],x1[l]-x1[l-1],0,color='black',lw=0.5,head_width=0.05,length_includes_head=True,zorder=100)
                plt.scatter(x1[l],x2[l],color='white',edgecolor='red',s=30,linewidth=1,alpha=1.0,zorder=100)
                plt.annotate(r'$f_{X_1|X_2}$ = ' + str(np.round(x2[l],2)),xy=[min(x[mask])-0.5,x2[l]+0.1],color='red')
    
    df = pd.DataFrame(np.vstack([x1,x2]).T, columns= ['x1','x2'])
    if L > 20:
        sns.kdeplot(data=df,x='x1',y='x2',color='grey',linewidths=1.0,alpha=min(((l-20)/20),1.0),levels=5,zorder=1)
    add_grid()
    plt.xlim([-3.5,3.5]); plt.ylim([-3.5,3.5]); plt.xlabel(r'$X_1$'); plt.ylabel(r'$X_2$'); plt.title('Gibbs Sampler - Bivariate Joint Gaussian Distribution')
    plt.subplots_adjust(left=0.0,bottom=0.0,right=1.0,top=1.1); plt.show() # set plot size 
    
# connect the function to make the samples and plot to the widgets    
interactive_plot = widgets.interactive_output(run_plot, {'nsample':nsample,'rho':rho})
interactive_plot.clear_output(wait = True)               # reduce flickering by delaying plot updating  

### Interactive Gibbs Sampler Demonstation 

#### Michael Pyrcz, Professor, The University of Texas at Austin 

Set the number of samples and correlation coefficient and observe the Gibbs sampler.

### The Inputs

* **$n_{sample}$** - number of samples, **$\rho_{X_1,X_2}$** - correlation coefficient

In [4]:
display(ui2, interactive_plot)                           # display the interactive plot

Output()

#### Comments

This was a basic demonstration of the Gibbs sampler for McMC. I have many other demonstrations and even basics of working with DataFrames, ndarrays, univariate statistics, plotting data, declustering, data transformations and many other workflows available at https://github.com/GeostatsGuy/PythonNumericalDemos and https://github.com/GeostatsGuy/GeostatsPy. 
  
#### The Author:

### Michael J. Pyrcz, Professor, The University of Texas at Austin 
*Novel Data Analytics, Geostatistics and Machine Learning Subsurface Solutions*

With over 17 years of experience in subsurface consulting, research and development, Michael has returned to academia driven by his passion for teaching and enthusiasm for enhancing engineers' and geoscientists' impact in subsurface resource development. 

For more about Michael check out these links:

#### [Twitter](https://twitter.com/geostatsguy) | [GitHub](https://github.com/GeostatsGuy) | [Website](http://michaelpyrcz.com) | [GoogleScholar](https://scholar.google.com/citations?user=QVZ20eQAAAAJ&hl=en&oi=ao) | [Book](https://www.amazon.com/Geostatistical-Reservoir-Modeling-Michael-Pyrcz/dp/0199731446) | [YouTube](https://www.youtube.com/channel/UCLqEr-xV-ceHdXXXrTId5ig)  | [LinkedIn](https://www.linkedin.com/in/michael-pyrcz-61a648a1)

#### Want to Work Together?

I hope this content is helpful to those that want to learn more about subsurface modeling, data analytics and machine learning. Students and working professionals are welcome to participate.

* Want to invite me to visit your company for training, mentoring, project review, workflow design and / or consulting? I'd be happy to drop by and work with you! 

* Interested in partnering, supporting my graduate student research or my Subsurface Data Analytics and Machine Learning consortium (co-PIs including Profs. Foster, Torres-Verdin and van Oort)? My research combines data analytics, stochastic modeling and machine learning theory with practice to develop novel methods and workflows to add value. We are solving challenging subsurface problems!

* I can be reached at mpyrcz@austin.utexas.edu.

I'm always happy to discuss,

*Michael*

Michael Pyrcz, Ph.D., P.Eng. Professor, The Hildebrand Department of Petroleum and Geosystems Engineering, Bureau of Economic Geology, Jackson School of Geosciences, The University of Texas at Austin

#### More Resources Available at: [Twitter](https://twitter.com/geostatsguy) | [GitHub](https://github.com/GeostatsGuy) | [Website](http://michaelpyrcz.com) | [GoogleScholar](https://scholar.google.com/citations?user=QVZ20eQAAAAJ&hl=en&oi=ao) | [Book](https://www.amazon.com/Geostatistical-Reservoir-Modeling-Michael-Pyrcz/dp/0199731446) | [YouTube](https://www.youtube.com/channel/UCLqEr-xV-ceHdXXXrTId5ig)  | [LinkedIn](https://www.linkedin.com/in/michael-pyrcz-61a648a1)  
  